In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from visualization_utilities import *
from metadata import *
from utilities2015 import *

Setting environment for Precision WorkStation


In [2]:
from pandas import *

# Check structure annotations

In [3]:
structures_df = DataManager.load_annotation_v4(stack='MD661', by_human=True, 
timestamp='latest', suffix='structures')

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD661" "/home/yuncong/CSHL_labelings_v3/MD661" --exclude "*" --include "*structures*"
0
latest timestamp:  02062018184326


In [37]:
def get_edit_transform(structures_df):
    for struct_id, structure_info in structures_df.iterrows():
        if structure_info['volume'] is not None:
            vol = bp.unpack_ndarray_str(structure_info['volume'])
            name_s = compose_label(structure_info['name'], side=structure_info['side'])
            print name_s, [ed['type'] for ed in structure_info['edits']]

            global_tf_from_wholebrain_to_wholebrain_volResol = np.eye(4)
            local_tf_from_wholebrain_to_wholebrain_volResol = np.eye(4)
            composed_tf_from_wholebrain_to_wholebrain_volResol = np.eye(4)
            for edit in structure_info['edits']:
                edit_tf_from_wholebrain_to_wholebrain_volRes = consolidate(edit['transform'])
                if edit['type'].startswith('global'): #  global_rotate3d or global_shift3d
                    global_tf_from_wholebrain_to_wholebrain_volResol = \
                    np.dot(edit_tf_from_wholebrain_to_wholebrain_volRes, global_tf_from_wholebrain_to_wholebrain_volResol)
                elif edit['type'].startswith('prob'):
                    local_tf_from_wholebrain_to_wholebrain_volResol = \
                    np.dot(edit_tf_from_wholebrain_to_wholebrain_volRes, local_tf_from_wholebrain_to_wholebrain_volResol)                    
                    
                composed_tf_from_wholebrain_to_wholebrain_volResol = \
                np.dot(edit_tf_from_wholebrain_to_wholebrain_volRes, composed_tf_from_wholebrain_to_wholebrain_volResol)

            print global_tf_from_wholebrain_to_wholebrain_volResol.reshape((4,4))[:3]
            print local_tf_from_wholebrain_to_wholebrain_volResol.reshape((4,4))[:3]
            print composed_tf_from_wholebrain_to_wholebrain_volResol.reshape((4,4))[:3]

In [38]:
get_edit_transform(structures_df)

Sp5C_L ['global_shift3d', 'global_rotate3d', 'global_rotate3d', 'global_shift3d', 'global_shift3d', 'prob_shift3d']
[[   0.96810387    0.25054921    0.         -112.26988376]
 [  -0.25054921    0.96810387    0.          218.69344179]
 [   0.            0.            1.          -16.56      ]]
[[  1.           0.           0.          32.71111111]
 [  0.           1.           0.         -18.4       ]
 [  0.           0.           1.           0.        ]]
[[   0.96810387    0.25054921    0.          -79.55877265]
 [  -0.25054921    0.96810387    0.          200.29344179]
 [   0.            0.            1.          -16.56      ]]
SNR_L ['global_shift3d', 'global_rotate3d', 'global_rotate3d', 'global_shift3d', 'global_shift3d']
[[   0.96810387    0.25054921    0.         -112.26988376]
 [  -0.25054921    0.96810387    0.          218.69344179]
 [   0.            0.            1.          -16.56      ]]
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]]
[[   0.96810387    0.250549

# Newer version

In [4]:
stack = 'MD589'

In [176]:
labeling_dir = '/home/yuncong/CSHL_labelings_v3/' + stack

In [177]:
annotations = {}

for fp in sorted(os.listdir(labeling_dir)):
    data = load_hdf_v2(os.path.join(labeling_dir, fp))
    print fp
    for ind, row in data.iterrows():
        print row['name'], row['section']
        annotations[(row['name'], row['section'])] = row    

CHATM3_annotation_contours_04162018144225.hdf
Pn 160
CHATM3_annotation_contours_04162018144658.hdf
Pn 160
7N 160
IC 160
SNC 160
5N 160
SNR 160
SC 160
CHATM3_annotation_contours_04162018144850.hdf
7n 160
5N 160
SNC 160
SC 160
Pn 160
Amb 160
LRt 160
SNR 160
7N 160
IC 160
CHATM3_annotation_contours_04162018145757.hdf
SNC 163
SNC 160
IC 163
7n 160
5N 164
SNR 160
SC 163
SC 160
7N 160
IC 160
Pn 166
LRt 160
5N 160
Pn 160
SNR 163
Amb 160
CHATM3_annotation_contours_04162018150333.hdf
LRt 160
IC 163
SC 163
5N 164
Pn 166
SNC 176
SNR 160
7N 160
LRt 163
5N 160
SNC 163
SNC 160
SNR 163
7n 160
SC 160
Pn 160
Amb 160
SNC 171
SNR 176
SNR 171
7N 170
IC 160
CHATM3_annotation_contours_04162018152327.hdf
RtTg 192
7n 192
SC 163
7N 170
SNR 176
SC 184
SNR 163
Tz 184
Tz 205
SC 200
IC 184
Pn 166
6N 204
RtTg 207
RtTg 184
Pn 160
SNC 176
SNR 182
LC 175
RtTg 204
Pn 207
RMC 192
RMC 184
Tz 192
RMC 207
RMC 200
IC 207
LRt 160
SC 160
SNR 171
6N 207
7n 200
LRt 163
5N 160
7n 160
IC 200
SNC 171
IC 160
Tz 207
6N 205
5N 164
SC

In [178]:
sorted(annotations.keys())

[('10N', 211),
 ('10N', 235),
 ('10N', 247),
 ('12N', 211),
 ('12N', 227),
 ('12N', 235),
 ('12N', 247),
 ('5N', 160),
 ('5N', 164),
 ('6N', 204),
 ('6N', 205),
 ('6N', 207),
 ('7N', 160),
 ('7N', 170),
 ('7n', 160),
 ('7n', 192),
 ('7n', 200),
 ('AP', 227),
 ('AP', 235),
 ('Amb', 160),
 ('IC', 160),
 ('IC', 163),
 ('IC', 184),
 ('IC', 200),
 ('IC', 207),
 ('IC', 219),
 ('IC', 227),
 ('LC', 175),
 ('LRt', 160),
 ('LRt', 163),
 ('Pn', 160),
 ('Pn', 166),
 ('Pn', 184),
 ('Pn', 192),
 ('Pn', 207),
 ('RMC', 184),
 ('RMC', 192),
 ('RMC', 200),
 ('RMC', 207),
 ('RMC', 235),
 ('RMC', 247),
 ('RtTg', 184),
 ('RtTg', 192),
 ('RtTg', 204),
 ('RtTg', 207),
 ('RtTg', 211),
 ('RtTg', 227),
 ('SC', 160),
 ('SC', 163),
 ('SC', 184),
 ('SC', 200),
 ('SC', 207),
 ('SC', 219),
 ('SC', 227),
 ('SNC', 160),
 ('SNC', 163),
 ('SNC', 171),
 ('SNC', 176),
 ('SNR', 160),
 ('SNR', 163),
 ('SNR', 171),
 ('SNR', 176),
 ('SNR', 182),
 ('Tz', 184),
 ('Tz', 192),
 ('Tz', 205),
 ('Tz', 207)]

In [179]:
merged_annotations_table = DataFrame(annotations.values())

In [180]:
merged_annotations_table

,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
61417973919587,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide83_2018_01_26-S2,61417973919587,"[45242.97334769556, 14458.899611503883]",10N,sagittal,[],raw,247,None,False,04162018154212,confirmed,"[[44355.51853031244, 14412.688257996695], [441..."
165498910605087,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide62_2018_02_19-S2,165498910605087,"[29560.777569723257, 16078.946725111617]",SNR,sagittal,[],raw,182,None,False,04162018151052,confirmed,"[[29458.24516597796, 16466.65042560262], [2915..."
110473719026558,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide66_2018_02_19-S1,110473719026558,"[34502.32822582653, 16157.73820339809]",RtTg,sagittal,[],raw,192,None,False,04162018151246,confirmed,"[[33282.32820358013, 15984.895809762857], [336..."
102790288330653,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide77_2018_01_26-S1,102790288330653,"[32229.89735960506, 6030.339041948793]",SC,sagittal,[],raw,227,None,False,04162018152920,confirmed,"[[31340.615265167744, 7394.858573806586], [309..."
108860727905611,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide77_2018_01_26-S1,108860727905611,"[35530.58904660281, 5787.660395539464]",IC,sagittal,[],raw,227,None,False,04162018152900,confirmed,"[[34730.80377764012, 5168.051308139539], [3468..."
131633584750329,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide70_2018_02_19-S1,131633584750329,"[34516.40779894652, 14570.824878276488]",6N,sagittal,[],raw,204,None,False,04162018151748,confirmed,"[[34248.00483041288, 14764.966211574136], [340..."
217866353503758,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide59_2018_02_19-S2,217866353503758,"[38807.11996040599, 10398.53216215258]",LC,sagittal,[],raw,175,None,False,04162018150357,confirmed,"[[38963.409612330135, 10057.157328010566], [38..."
146476015747094,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide69_2018_02_19-S2,146476015747094,"[33214.52316272273, 13546.70416079169]",6N,sagittal,[],raw,205,None,False,04162018151919,confirmed,"[[32923.39783832048, 13734.518908083017], [328..."
126673529461145,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide62_2018_02_19-S3,126673529461145,"[36777.61878491937, 18569.784908726448]",Tz,sagittal,[],raw,184,None,False,04162018150824,confirmed,"[[36280.710727115766, 18934.772876054507], [35..."
65942681262602,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM3_slide54_2018_02_18-S3,65942681262602,"[32927.618200539524, 7901.654884485934]",SC,sagittal,[],raw,160,None,False,04162018144513,confirmed,"[[32214.71748645316, 10149.322529429843], [316..."


In [181]:
fp = DataManager.get_annotation_filepath(stack=stack, by_human=True, suffix='contours', timestamp='now')
save_hdf_v2(merged_annotations_table, fp)
upload_to_s3(fp)

aws s3 cp "/home/yuncong/CSHL_labelings_v3/CHATM3/CHATM3_annotation_contours_04172018132221.hdf" "s3://mousebrainatlas-data/CSHL_labelings_v3/CHATM3/CHATM3_annotation_contours_04172018132221.hdf"
0


In [183]:
contours_df, timestamp = DataManager.load_annotation_v4(stack=stack, by_human=True, timestamp='latest', 
                                                        suffix='contours', return_timestamp=True)

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/CHATM2" "/home/yuncong/CSHL_labelings_v3/CHATM2" --exclude "*" --include "*contours*"
0
latest timestamp:  04172018132213


In [184]:
contours_df

,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
187424345944670,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide88_2018_02_13-S2,187424345944670,"[22449.138208354954, 7849.2298064949855]",SC,sagittal,[],raw,265,None,False,04162018162955,confirmed,"[[21193.799839839758, 7489.076651597285], [217..."
131542146410241,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide92_2018_02_13-S2,131542146410241,"[25794.57562654516, 6979.149702085063]",IC,sagittal,[],raw,277,None,False,04162018163343,confirmed,"[[25403.093933940287, 7503.605559214472], [262..."
258771944424632,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide92_2018_02_13-S2,258771944424632,"[21927.129970996564, 11451.427401169789]",SNR,sagittal,[],raw,277,None,False,04162018163147,confirmed,"[[21512.516630547132, 12117.22259035926], [212..."
206211808342086,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide80_2018_02_14-S2,206211808342086,"[20858.499838983465, 13697.70086692388]",SNC,sagittal,[],raw,241,None,False,04162018161108,confirmed,"[[20500.167651701908, 14130.013443023021], [20..."
43110298657518,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide88_2018_02_13-S2,43110298657518,"[25776.45683248748, 6714.420874910297]",IC,sagittal,[],raw,265,None,False,04162018162918,confirmed,"[[25779.367261136336, 5360.013552921586], [259..."
36601380358710,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide88_2018_02_13-S2,36601380358710,"[21290.35235834303, 13223.458809884298]",SNR,sagittal,[],raw,265,None,False,04162018162854,confirmed,"[[20159.57623090962, 14255.37163974729], [2056..."
276240472044097,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide54_2018_01_22-S2,276240472044097,"[29631.50440693211, 14252.74772861855]",6N,sagittal,[],raw,160,None,False,04162018154614,confirmed,"[[29371.115451339174, 14434.062942408422], [29..."
34196477312018,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide92_2018_02_13-S2,34196477312018,"[24304.05427557077, 9382.26552343177]",PBG,sagittal,[],raw,277,None,False,04162018163305,confirmed,"[[23739.854681531353, 10275.34006501101], [234..."
3350986096957,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide54_2018_01_22-S2,3350986096957,"[25351.49154941229, 5679.344426991671]",SC,sagittal,[],raw,160,None,False,04162018154717,confirmed,"[[24909.17604135787, 4095.962733666653], [2538..."
242216048696653,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'041...",CHATM2_slide80_2018_02_14-S2,242216048696653,"[21912.691006048422, 14109.434007245887]",SNR,sagittal,[],raw,241,None,False,04162018161049,confirmed,"[[21528.201705224245, 14905.282111688157], [20..."


# New version

In [5]:
contours_df, timestamp = DataManager.load_annotation_v4(stack='MD589', by_human=True, timestamp='latest', 
                                                        suffix='contours', return_timestamp=True)

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/home/yuncong/CSHL_labelings_v3/MD589" --exclude "*" --include "*contours*"
0
latest timestamp:  04192018111821


In [6]:
contours_df

,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
100079545430812,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-N24-2015.07.30-17.35.51_MD589_3_0072,100079545430812,"[27955.713503430972, 14489.042075289639]",Sp5C,sagittal,[],raw,140,L,False,09122016153534,confirmed,"[[27336.28671765756, 13673.240742902675], [275..."
10032954325690,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-N26-2015.07.30-17.44.37_MD589_2_0077,10032954325690,"[20223.593755908256, 11905.6849509605]",SNR,sagittal,[],raw,150,L,False,09122016163509,confirmed,"[[19109.352139599978, 12512.51694693348], [190..."
10042629543436,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-N54-2015.07.30-19.44.32_MD589_1_0160,10042629543436,"[25128.495007919257, 15759.047158120664]",7N,sagittal,[],raw,276,R,False,09152016032103,confirmed,"[[25216.1972767153, 15950.633462558133], [2505..."
100499768892293,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-N49-2015.07.30-19.20.15_MD589_1_0145,100499768892293,"[21977.27533171153, 14617.706050584224]",Pn,sagittal,[],raw,250,R,False,09122016210517,confirmed,"[[21199.849905415373, 13913.002831062433], [21..."
100567129513785,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-IHC40-2015.07.31-00.02.21_MD589_2_0119,100567129513785,"[29674.80044267275, 15845.32551451423]",10N,sagittal,[],raw,217,L,False,09152016013529,confirmed,"[[30223.781101150613, 16378.173473763134], [30..."
100571878935576,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-IHC31-2015.07.30-23.26.22_MD589_2_0092,100571878935576,"[29426.045475844207, 17409.091722532714]",LRt,sagittal,[],raw,181,L,False,09152016011716,confirmed,"[[29944.527125099856, 17807.829483902246], [29..."
100716602659441,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-IHC49-2015.07.31-00.39.15_MD589_1_0145,100716602659441,"[21335.19092039978, 11591.502132341939]",RMC,sagittal,[],raw,251,R,False,09152016021650,confirmed,"[[21071.674204280214, 12031.051763893447], [20..."
10076160352035,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-IHC44-2015.07.31-00.18.50_MD589_1_0130,10076160352035,"[26364.418453306247, 7813.769347082874]",IC,sagittal,[],raw,231,S,False,09152016012134,confirmed,"[[26219.203919319396, 8854.855353984922], [261..."
100898193708403,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-N43-2015.07.30-18.56.33_MD589_2_0128,100898193708403,"[26161.34262628827, 7500.26019166984]",IC,sagittal,[],raw,228,S,False,09152016012134,confirmed,"[[25969.585030560545, 8466.21123574113], [2589..."
100909285842903,contour,yuncong,"[{u'username': u'yuncong', u'timestamp': u'091...",MD589-N44-2015.07.30-19.00.35_MD589_2_0131,100909285842903,"[30985.519773329503, 14500.648334735612]",10N,sagittal,[],raw,232,R,False,09152016020848,confirmed,"[[31403.69804250788, 14893.95935225501], [3129..."


In [9]:
contours_df = contours_df[(contours_df['filename'] != 'Placeholder') & (contours_df['name'] == '7N')]

In [6]:
contours_df['side'][contours_df['name'] == '12N'] = 'S'

In [7]:
contours_df[(contours_df['type'] == 'confirmed') & (contours_df['name'] == '12N')]

,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices


In [24]:
sagittal_contours_df_fp = DataManager.get_annotation_filepath(stack='MD658', by_human=True, suffix='contours', 
                                                              timestamp='now')
save_hdf_v2(contours_df, sagittal_contours_df_fp)
upload_to_s3(sagittal_contours_df_fp)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->['class', 'creator', 'downsample', 'edits', 'filename', 'id', 'label_position', 'name', 'orientation', 'parent_structure', 'section', 'side', 'side_manually_assigned', 'time_created', 'type', 'vertices']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


aws s3 cp "/home/yuncong/CSHL_labelings_v3/MD658/MD658_annotation_contours_01082018162524.hdf" "s3://mousebrainatlas-data/CSHL_labelings_v3/MD658/MD658_annotation_contours_01082018162524.hdf"


# Old version

In [1]:
stack = 'MD635'

In [6]:
fp = '/home/yuncong/CSHL_labelings_v3/%(stack)s/%(stack)s_annotation_v3.h5' % {'stack': stack}
download_from_s3(fp)
contours = read_hdf(fp)

rm -rf "/home/yuncong/CSHL_labelings_v3/MD635/MD635_annotation_v3.h5" && mkdir -p "/home/yuncong/CSHL_labelings_v3/MD635"
aws s3 cp "s3://mousebrainatlas-data/CSHL_labelings_v3/MD635/MD635_annotation_v3.h5" "/home/yuncong/CSHL_labelings_v3/MD635/MD635_annotation_v3.h5"


In [12]:
contours[(contours['name'] == '7N') & (contours['section'] == 158)]

,creator,downsample,edits,filename,flags,id,label_position,name,orientation,parent_structure,section,side,side_manually_assigned,time_created,vertices
102627349379154,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'122...",MD635-F53-2016.05.19-05.14.50_MD635_2_0158,1,102627349379154,"[25175.1667763, 12351.1854336]",7N,sagittal,[],158,L,False,12262016143425,"[[24780.2371365, 12783.8648579], [24620.295638..."
106833387670490,yuncong,1,"[{u'username': u'yuncong', u'timestamp': u'122...",MD635-F53-2016.05.19-05.14.50_MD635_2_0158,1,106833387670490,"[25175.1667763, 12351.1854336]",7N,sagittal,[],158,L,False,12262016141652,"[[24780.2371365, 12783.8648579], [24620.295638..."


In [7]:
from annotation_utilities import *

In [8]:
contours = contours[(contours['orientation'] == 'sagittal') & (contours['downsample'] == 1)]
contours = convert_annotation_v3_original_to_aligned_cropped(contours, stack=stack)

In [10]:
grouped = contours.groupby('section')

In [15]:
for sec, group in grouped:
    for contour_id, contour in group.iterrows():
        print contour_id
    print '\n'

24041485827291


178638821776884


225468065711699


82753828715844


108639196063443
240450955921646


139995532033938
276274803125745


123201065869455
50599366457376


128891739813762
280008487571593
74734208874016


137576545852286
139884971202912
44979814970993


198648392198851
252434421302035
53849871852705


215403441631652
272974593078035
32236200064177


165920538830976
243256546173591
58906621711441


175994136617351
258516032496448
45531577507434


187863059163195
193809479990018
245020993386827


124244720326548
28990358901640
6039163938366


13783358246531
269642137367206
93951936437221


170927976283983
250086639456848
65686469868581


12994163947428
131268714517286
180619860209674
3348124243740


140881450963675
171250512966690
277597751075951
90654685894305


124892661563969
202987135024252
207964402507211
50422758984114
63049897456070


236059505041962
259084303736308
69129898116566
86007295995378
95926547893398


111731759580942
135434889301284
265153680649971
273505